In [23]:
import pandas as pd
from bs4 import BeautifulSoup


# SOURCES:
# https://stackoverflow.com/questions/53189494/apply-beautifulsoup-function-to-pandas-dataframe
# https://stackoverflow.com/questions/20297317/python-dataframe-pandas-drop-column-using-int
# Data Cleaning Notebook from Week 2 
# https://stackoverflow.com/questions/17383094/how-can-i-map-true-false-to-1-0-in-a-pandas-dataframe
# https://stackoverflow.com/questions/39885359/beautifulsoup-decompose

def has_tag(df, cols, tag):
    df_code = pd.DataFrame() 
    
    for c in cols:
        code = df[c].apply(lambda text: BeautifulSoup(text, 'html.parser').find_all(tag))
        col_name = c + '_HAS_CODE'
        
        code = (code.str.len() > 0).astype(int).rename(col_name).to_frame()
        df_code = pd.concat([df_code,code], axis=1)

    return df_code


def decompose_html(text, tag):   
    soup = BeautifulSoup(text, 'html.parser')
    code_tags = soup.findAll(tag)

    for code in code_tags:
        code.decompose()
    
    return soup


def alter_col_text(df, cols, tag):   
    for c in cols:
        df[c] = df[c].apply(lambda text: decompose_html(text,tag)) 
    return df


df = pd.read_csv('stack_data.csv',header=0)
df.drop(df.columns[2], axis=1, inplace=True)
df.fillna('', inplace=True)

cols = ['QUESTION_BODY','ACCEPTED_ANSWER_BODY']
tag = 'code'


df = pd.concat([df,has_tag(df, cols, tag)], axis=1)

df = alter_col_text(df, ['QUESTION_BODY','ACCEPTED_ANSWER_BODY'],'code')

df.to_csv('out.csv')

